#### Batch query sales totals from Kafka in Avro format
* __Author:__  Gary A. Stafford  
* __Date:__ 2021-10-03  
* __Post:__ [Stream Processing with Apache Spark, Kafka, Avro, and Apicurio Registry on Amazon EMR and Amazon MSK](https://itnext.io/stream-processing-with-apache-spark-kafka-avro-and-apicurio-registry-on-amazon-emr-and-amazon-13080defa3be)  
* __Description:__  
Notebook version of `13_batch_read_results_avro.py`. Script performs a batch query of all the Avro-format aggregated sales messages from the Kafka topic, `pagila.sales.summary.avro`, using schemas stored in Apicurio Registry. The script then summarizes the final sales results for each sliding 10-minute event-time window, by sales region.

In [ ]:
%%spark

In [ ]:
import os

import boto3
import pyspark.sql.functions as F
import requests
from ec2_metadata import ec2_metadata
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro
from pyspark.sql.window import Window

In [ ]:
def get_schema(artifact_id):
    """Get Avro schema from Apicurio Registry"""

    response = requests.get(
        f"{params['schema_registry_url']}/apis/registry/v2/groups/default/artifacts/{artifact_id}"
    )
    json_format_schema = response.content.decode("utf-8")

    return json_format_schema


def get_parameters():
    """Load parameter values from AWS Systems Manager (SSM) Parameter Store"""

    parameters = {
        "kafka_servers":
            ssm_client.get_parameter(Name="/kafka_spark_demo/kafka_servers")
            ["Parameter"]["Value"],
        "kafka_demo_bucket":
            ssm_client.get_parameter(Name="/kafka_spark_demo/kafka_demo_bucket")
            ["Parameter"]["Value"],
        "schema_registry_url":
            ssm_client.get_parameter(
                Name="/kafka_spark_demo/schema_registry_url_int")["Parameter"]
            ["Value"],
    }

    return parameters

In [ ]:
source_topic = "pagila.sales.summary.avro"

os.environ['AWS_DEFAULT_REGION'] = ec2_metadata.region
ssm_client = boto3.client("ssm")
params = get_parameters()

In [ ]:
# retrieve schemas from registry

sales_summary_key = get_schema("pagila.sales.summary.avro-key")
sales_summary_value = get_schema("pagila.sales.summary.avro-value")

In [ ]:
# batch query kafka topic

options_read = {
    "kafka.bootstrap.servers":
        params["kafka_servers"],
    "subscribe":
        source_topic,
    "startingOffsets":
        "earliest",
    "failOnDataLoss":
        "false",
    "kafka.ssl.truststore.location":
        "/tmp/kafka.client.truststore.jks",
    "kafka.security.protocol":
        "SASL_SSL",
    "kafka.sasl.mechanism":
        "AWS_MSK_IAM",
    "kafka.sasl.jaas.config":
        "software.amazon.msk.auth.iam.IAMLoginModule required;",
    "kafka.sasl.client.callback.handler.class":
        "software.amazon.msk.auth.iam.IAMClientCallbackHandler"
}

df_sales = spark.read \
    .format("kafka") \
    .options(**options_read) \
    .load() \
    .select("timestamp",
            from_avro("key", sales_summary_key).alias("key"),
            from_avro("value", sales_summary_value).alias("data")) \
    .select("timestamp", "key", "data.*")

In [ ]:
df_sales.summary().show()

In [ ]:
df_sales.printSchema()

In [ ]:
%%display -n 10
df_sales

In [ ]:
# raw aggregated sales messages (kafka message value)

df_sales \
    .drop("timestamp", "key") \
    .show(25, truncate=False)

In [ ]:
# example of one region's aggregated sales values (Kafka messages)

df_sales \
    .filter(F.col("region") == "Asia & Pacific") \
    .select(F.date_format("timestamp", format="yyyy-MM-dd HH:mm").alias("timestamp"),
            F.col("region").alias("sales_region"),
            F.format_number("sales", 2).alias("sales"),
            F.format_number("orders", 0).alias("orders"),
            F.from_unixtime("window_start", format="yyyy-MM-dd HH:mm").alias("window_start"),
            F.from_unixtime("window_end", format="yyyy-MM-dd HH:mm").alias("window_end")) \
    .orderBy(F.col("window_end").desc(), F.col("timestamp").desc()) \
    .show(30, truncate=False)

In [ ]:
# example of single sales region's aggregated sales values during single 10-minute event window

df_sales \
    .select(F.date_format("timestamp", format="yyyy-MM-dd HH:mm").alias("timestamp"),
            F.col("region").alias("sales_region"),
            F.format_number("sales", 2).alias("sales"),
            F.format_number("orders", 0).alias("orders"),
            F.from_unixtime("window_start", format="yyyy-MM-dd HH:mm").alias("window_start"),
            F.from_unixtime("window_end", format="yyyy-MM-dd HH:mm").alias("window_end")) \
    .filter(F.col("region") == "Asia & Pacific") \
    .filter(F.col("window_start") == "2021-10-03 22:25") \
    .orderBy(F.col("timestamp").desc()) \
    .show(truncate=False)

In [ ]:
# get final sales for each region, by event-time window using Window.partitionBy and row_number().over(window)

window = Window.partitionBy("region",
                            "window_start").orderBy(F.col("timestamp").desc())

df_sales_summary = df_sales \
    .withColumn("row", F.row_number().over(window)) \
    .filter(F.col("row") == 1).drop("row") \
    .select(F.col("region").alias("sales_region"),
            F.format_number("sales", 2).alias("sales"),
            F.format_number("orders", 0).alias("orders"),
            F.from_unixtime("window_start", format="yyyy-MM-dd HH:mm").alias("window_start"),
            F.from_unixtime("window_end", format="yyyy-MM-dd HH:mm").alias("window_end")) \
    .orderBy(F.col("window_start").desc(),
             F.regexp_replace("sales", ",", "").cast("float").desc())

In [ ]:
df_sales_summary.summary().show()

In [ ]:
df_sales_summary.printSchema()

In [ ]:
df_sales_summary.show(36, truncate=False)
